In [5]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt


In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
input_size = 784
hidden_size = 100
num_classes = 10
num_epochs = 2
batch_size = 100
learning_rate = 0.001

In [8]:
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, 
                                  transform=transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, 
                                  transform=transforms.ToTensor(), download=True)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size , shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size , shuffle=False)

examples = iter(train_loader)
samples , label = next(examples)
print(samples.shape, label.shape)


torch.Size([100, 1, 28, 28]) torch.Size([100])


In [9]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()   
        self.l1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        return out

model = NeuralNet(input_size, hidden_size, num_classes)

In [10]:
model

NeuralNet(
  (l1): Linear(in_features=784, out_features=100, bias=True)
  (relu): ReLU()
  (l2): Linear(in_features=100, out_features=10, bias=True)
)

In [11]:
#loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [15]:
for epoch in range(len(train_loader)):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1,28*28).to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'epoch {epoch+1} / {num_epochs}, step {i+1}, loss = {loss.item()}')

    with torch.no_grad():
        n_correct = 0
        n_samples = 0
        for images, labels in test_loader:
            images = images.reshape(-1, 28*28)
            labels = labels.to(device)
            outputs = model(images)

            _, predictions = torch.max(outputs, 1)
            n_samples +=labels.shape[0]
            n_correct +=(predictions == labels).sum().item()
        acc = 100.0 * (n_correct / n_samples)
        print(f'accuracy = {acc}')
        

epoch 1 / 2, step 100, loss = 0.021458700299263
epoch 1 / 2, step 200, loss = 0.08079078793525696
epoch 1 / 2, step 300, loss = 0.08658201992511749
epoch 1 / 2, step 400, loss = 0.06788129359483719
epoch 1 / 2, step 500, loss = 0.08358700573444366
epoch 1 / 2, step 600, loss = 0.06421217322349548
accuracy = 97.68
epoch 2 / 2, step 100, loss = 0.04497966170310974
epoch 2 / 2, step 200, loss = 0.03019811399281025
epoch 2 / 2, step 300, loss = 0.08864017575979233
epoch 2 / 2, step 400, loss = 0.030997226014733315
epoch 2 / 2, step 500, loss = 0.05571833252906799
epoch 2 / 2, step 600, loss = 0.0606037899851799
accuracy = 97.50999999999999
epoch 3 / 2, step 100, loss = 0.030753765255212784
epoch 3 / 2, step 200, loss = 0.03803838789463043
epoch 3 / 2, step 300, loss = 0.028999121859669685
epoch 3 / 2, step 400, loss = 0.03430977836251259
epoch 3 / 2, step 500, loss = 0.06471449136734009
epoch 3 / 2, step 600, loss = 0.04773830994963646
accuracy = 97.56
epoch 4 / 2, step 100, loss = 0.04046

KeyboardInterrupt: 